# Лаба 3

**Дедлайн**: 9 декабря

**Задача**: разобраться в описанном ниже коде, используемой/используемых моделях, ответить на указанные вопросы и сгенерировать faceswap со своей фотографией

**Описание**

Порой, вместо самостоятельного написания нейросети и ее обучения (или дообучения) приходится использовать уже готовые модели и разбираться, как они устроены - будь то код в репозитории на Github-e, Jupyter Notebook или специальная библиотека с небольшой документацией. В данном задании вам предлагается решить эту задачу - разобраться с уже готовой моделью, успешно запустить ее, залезть в сорсы и посмотреть, как крупная модель устроена изнутри.

В качестве решения лабораторной ожидается отчет в конце этого Jupyter Notebook-a с результатами генерации по вашим изображениям и ответами на следующие вопросы:

1. Где расположен код модели и кто разработчик?
2. Есть ли статья по работе модели? Если да, то в чем суть подхода авторов, как обучалась модель, какие метрики были достигнуты?
3. Состоит ли модель из нескольких частей? Если да, то из каких, что они делают и как называются в коде?
4. Как проходит процесс инференса модели для фото и видео (подробно)? Какие применяются преобразования входных и выходных данных?
5. Можно ли дообучить модель и какие для этого требуются ресурсы? Может, авторы давали советы по обучению/дообучению? 

# GHOST: Generative High-fidelity One Shot Transfer

<img src="https://drive.google.com/uc?export=view&id=1MdOWxP9CqyqmW6t9MQ6-gcfEux54zbqP">

In [1]:
#@markdown #**Check GPU and CUDA version**

!nvidia-smi

!nvcc --version

Mon May  8 01:06:03 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 531.18                 Driver Version: 531.18       CUDA Version: 12.1     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                      TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf            Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 3060 Ti    WDDM | 00000000:26:00.0  On |                  N/A |
| 30%   37C    P8               23W / 200W|   1667MiB /  8192MiB |      2%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

'nvcc' is not recognized as an internal or external command,
operable program or batch file.


In [2]:
#@markdown #**Clone github & download models**

# !git clone https://github.com/sberbank-ai/sber-swap.git
%cd sber-swap

# # load arcface
# !wget -P ./arcface_model https://github.com/sberbank-ai/sber-swap/releases/download/arcface/backbone.pth
# !wget -P ./arcface_model https://github.com/sberbank-ai/sber-swap/releases/download/arcface/iresnet.py
#
# # load landmarks detector
# !wget -P ./insightface_func/models/antelope https://github.com/sberbank-ai/sber-swap/releases/download/antelope/glintr100.onnx
# !wget -P ./insightface_func/models/antelope https://github.com/sberbank-ai/sber-swap/releases/download/antelope/scrfd_10g_bnkps.onnx
#
# # load model itself
# !wget -P ./weights https://github.com/sberbank-ai/sber-swap/releases/download/sber-swap-v2.0/G_unet_2blocks.pth
#
# # load super res model
# !wget -P ./weights https://github.com/sberbank-ai/sber-swap/releases/download/super-res/10_net_G.pth

C:\Users\anast3t\Desktop\all\coding\neural-networks\Task3\sber-swap


In [3]:
#@markdown #**Install required libraries**

# !pip install mxnet-cu112
# !pip install onnxruntime-gpu==1.8
# !pip install insightface==0.2.1
# !pip install kornia==0.5.4

In [4]:
#@markdown #**Preparation**

import cv2
import torch
import time
import os



from utils.inference.image_processing import crop_face, get_final_image, show_images
from utils.inference.video_processing import read_video, get_target, get_final_video, add_audio_from_another_video, face_enhancement
from utils.inference.core import model_inference

from network.AEI_Net import AEI_Net
from coordinate_reg.image_infer import Handler
from insightface_func.face_detect_crop_multi import Face_detect_crop
from arcface_model.iresnet import iresnet100
from models.pix2pix_model import Pix2PixModel
from models.config_sr import TestOptions

C:\Users\anast3t\miniconda3\envs\task3-face-swap\lib\site-packages\kornia\augmentation\augmentation.py:1833: DeprecationWarning: GaussianBlur is no longer maintained and will be removed from the future versions. Please use RandomGaussianBlur instead.
  category=DeprecationWarning,


In [ ]:
#@markdown #**Initialize models**

app = Face_detect_crop(name='antelope', root='./insightface_func/models')
app.prepare(ctx_id= 0, det_thresh=0.6, det_size=(640,640))

# main model for generation
G = AEI_Net(backbone='unet', num_blocks=2, c_id=512)
G.eval()
G.load_state_dict(torch.load('weights/G_unet_2blocks.pth', map_location=torch.device('cpu')))
G = G.cuda()
G = G.half()

# arcface model to get face embedding
netArc = iresnet100(fp16=False)
netArc.load_state_dict(torch.load('arcface_model/backbone.pth'))
netArc=netArc.cuda()
netArc.eval()

# model to get face landmarks
handler = Handler('./coordinate_reg/model/2d106det', 0, ctx_id=0, det_size=640)

# model to make superres of face, set use_sr=True if you want to use super resolution or use_sr=False if you don't
use_sr = True
if use_sr:
    os.environ['CUDA_VISIBLE_DEVICES'] = '0'
    torch.backends.cudnn.benchmark = True
    opt = TestOptions()
    #opt.which_epoch ='10_7'
    model = Pix2PixModel(opt)
    model.netG.train()

In [ ]:
source_full = cv2.imread("/content/ПАЛЬЧУНОВ.png")
source_full

In [ ]:
#@markdown #**Upload source image and video**

#@markdown choose not really long videos, coz it can take a lot of time otherwise  

#@markdown choose source image as a photo -- preferable a selfie of a person

target_type = 'image' #@param ["video", "image"]


source_path = '/content/ПАЛЬЧУНОВ.png' #@param {type:"string"}
target_path = '/content/Маск.jpeg' #@param {type:"string"}
path_to_video = '/content/nggyup.mp4' #@param {type:"string"}

source_full = cv2.imread(source_path)
OUT_VIDEO_NAME = "examples/results/result.mp4"
crop_size = 224 # don't change this


# check, if we can detect face on the source image

try:    
    source = crop_face(source_full, app, crop_size)[0]
    source = [source[:, :, ::-1]]
    print("Everything is ok!")
except TypeError:
    print("Bad source images")

# read video
if target_type == 'image':
    target_full = cv2.imread(target_path)
    full_frames = [target_full]
else:
    full_frames, fps = read_video(path_to_video)
target = get_target(full_frames, app, crop_size)

In [ ]:
#@markdown #**Inference**


batch_size =  40#@param {type:"integer"}

START_TIME = time.time()

final_frames_list, crop_frames_list, full_frames, tfm_array_list = model_inference(full_frames,
                                                                                   source,
                                                                                   target,
                                                                                   netArc,
                                                                                   G,
                                                                                   app,
                                                                                   set_target = False,
                                                                                   crop_size=crop_size,
                                                                                   BS=batch_size)

if use_sr:
    final_frames_list = face_enhancement(final_frames_list, model)

if target_type == 'video':
  get_final_video(final_frames_list,
                  crop_frames_list,
                  full_frames,
                  tfm_array_list,
                  OUT_VIDEO_NAME,
                  fps, 
                  handler)
  
  add_audio_from_another_video(path_to_video, OUT_VIDEO_NAME, "audio")

  print(f'Full pipeline took {time.time() - START_TIME}')
  print(f"Video saved with path {OUT_VIDEO_NAME}")
else:
  result = get_final_image(final_frames_list, crop_frames_list, full_frames[0], tfm_array_list, handler)
  cv2.imwrite('examples/results/result.png', result)

In [ ]:
#@markdown #**Visualize Image to Image swap**

import matplotlib.pyplot as plt

show_images([source[0][:, :, ::-1], target_full, result], ['Source Image', 'Target Image', 'Swapped Image'], figsize=(20, 15))


In [ ]:
#@markdown #**Visualize Video Swap**

from IPython.display import HTML
from base64 import b64encode

video_file = open(OUT_VIDEO_NAME, "r+b").read()
video_url = f"data:video/mp4;base64,{b64encode(video_file).decode()}"

HTML(f"""<video width={800} controls><source src="{video_url}"></video>""")